In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
file = 'ANZ_transactions.xlsx'
transactions = pd.read_excel(f'./data/{file}')

---

## Objective: `Targeted Marketing of Housing Loans to Customers`

---

### Data Inspection

In [ ]:
transactions.head(3)

In [ ]:
info = pd.DataFrame({
   'column': transactions.columns, 
   'total_null': transactions.isna().sum().values, 
   'na_%': np.round((transactions.isna().sum() / len(transactions)).values * 100, 2),
   'type': transactions.dtypes.values
}, index=range(1, transactions.shape[1] + 1)
)

In [ ]:
info.column

In [ ]:
info[info.total_null != 0]

Columns like `bpay_biller_code` and `merchant_code` have high null rate. We need to dig a bit further

In [ ]:
print('Dimension: ', transactions.shape)

---

### Descriptive Statistics

In [ ]:
transactions.describe()

---

## Exploratory Data Analysis

In [ ]:
features = {
   'categorical': ['status', 'currency', 'txn_description', 'gender', 'merchant_suburb', 'merchant_state', 'country', 'movement'],
   'numerical': ['card_present_flag', 'balance', 'age', 'amount']
}

### Categorical Features

In [ ]:
def value_counts_plot(feature: str, df: pd.DataFrame = transactions):
   """Plot a bar graph for the categorical feature <feature> in descending
   order and another bar graph for the average transaction amount by <feature>.
   Limit the plot to top 10 values.
   """
   fig, axes = plt.subplots(1, 2, figsize=(12,6))
   
   sns.countplot(data=df, x=feature, ax=axes[0]);
   sns.barplot(data=df, x=feature, y='amount', ax=axes[1])

   axes[0].set_title('Number of Transactions by ' + feature.capitalize())
   axes[1].set_title('Average Transaction Amount by ' + feature.capitalize())
   axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45)
   axes[1].set_xticklabels(axes[0].get_xticklabels(), rotation=45)
   

`Status`

In [ ]:
transactions.status.value_counts(dropna=False)

In [ ]:
value_counts_plot(feature='status')

`Currency`

In [ ]:
transactions.currency.value_counts(dropna=False)

`Transaction description`

In [ ]:
transactions.txn_description.value_counts(dropna=False)

In [ ]:
value_counts_plot(feature='txn_description')

`Gender`

In [ ]:
transactions.gender.value_counts(dropna=False)

In [ ]:
value_counts_plot(feature='gender')

`Merchant Suburb`

In [ ]:
transactions.merchant_suburb.value_counts(dropna=False)

Most transactions were processed in `Melbourne` and `Sydney`

`Merchant State`

In [ ]:
transactions.merchant_state.value_counts(dropna=False)

In [ ]:
value_counts_plot(feature='merchant_state')

`Country`

In [ ]:
transactions.country.value_counts(dropna=False)

`Movement`: Method of payment

In [ ]:
transactions.movement.value_counts(dropna=False)

In [ ]:
value_counts_plot(feature='movement')

There are transactions of a much higher average amount processed through credit. Should look more into this to see if there is any suspicious or fraud. 

`currency` and `country` show that all transactions are made in Australia in Australian dollar, that is a trivial fact. We may drop these two columns since they don't provide any additional information. 

In [4]:
if 'currency' in transactions.columns and 'country' in transactions.columns:
   del transactions['currency']
   del transactions['country']

In [ ]:
transactions.shape

### Numerical Features

In [ ]:
transactions.card_present_flag.value_counts(dropna=False)

How is balance distributed?

In [ ]:
sns.histplot(data=transactions, x='balance', kde=True, bins=30);
plt.title('Distribution of Customer Account Balance');

How is customer age distributed?

In [ ]:
sns.histplot(data=transactions, x='age', kde=True);
plt.title('Distribution of Customer Age');

`There seems to be two groups of customers: one group of young adult (~23) and another group of middle-aged adult (~40)`

How is transaction amounts distributed?

In [ ]:
sns.histplot(data=transactions, x='amount', kde=True);
plt.title('Distribution of Transaction Amount');

### Handling Missing Values

In [ ]:
info[info['total_null'] > 0].sort_values('na_%', ascending=False)

`merchant_code` and `bpay_biller_code` contain too much missing values so they serve little use to the following analysis. Let's drop them both.

In [5]:
if 'merchant_code' in transactions.columns:
   del transactions['merchant_code']
if 'bpay_biller_code' in transactions.columns:
   del transactions['bpay_biller_code']

Recall that some transactions aren't made via POS and some are made through phone call or bank payment. Could these non-POS transactions be exactly those where the merchant and card info are missing? 

In [ ]:
transactions.txn_description.unique()

In [ ]:
transactions[(transactions.txn_description != 'SALES-POS') & (transactions.txn_description != 'POS')].isna().sum().sort_values(ascending=False)

Exactly! These 4326 transactions are all non-POS transactions. They are still valid records so let's just impute them with NA values

In [6]:
impute_columns = ['merchant_state', 'merchant_id', 'merchant_long_lat', 'merchant_suburb', 'card_present_flag']

In [7]:
for col in impute_columns:
   transactions[col].fillna('N/A', inplace=True)

Double check that there is no null values

In [8]:
transactions.isna().sum()

status               0
card_present_flag    0
account              0
long_lat             0
txn_description      0
merchant_id          0
first_name           0
balance              0
date                 0
gender               0
age                  0
merchant_suburb      0
merchant_state       0
extraction           0
amount               0
transaction_id       0
customer_id          0
merchant_long_lat    0
movement             0
dtype: int64

## Feature Engineering

In [9]:
if 'month' not in transactions.columns:
   transactions['month'] = transactions.date.dt.month
if 'weekday' not in transactions.columns:
   transactions['weekday'] = transactions.date.dt.weekday
if 'month_name' not in transactions.columns:
   transactions['month_name'] = transactions.date.dt.month_name()
if 'day_name' not in transactions.columns:
   transactions['day_name'] = transactions.date.dt.day_name()

How many transactions do customers make each month on average? What's the average amount of transactions made each month?

In [ ]:
transaction_month = transactions.groupby('month', as_index=False)
pd.DataFrame({
   'month': transaction_month.indices.keys(),
   'total_number': transaction_month.size()['size'],
   'average_amount': transaction_month.aggregate({'amount': 'mean'})['amount']
})

How many transactions do customers make each month on average?

In [ ]:
monthly_average = transactions.groupby('customer_id').amount.sum().sort_values(ascending=False) / 3

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(data=monthly_average.to_frame(), x='amount', kde=True, bins=10);
plt.title('Customers\' Average Montly Transactions Amount');

How many transactions do customers make each week day on average? What's the average transaction amount made each day?

In [ ]:
transaction_day = transactions.groupby('day_name', as_index=False)
pd.DataFrame({
   'day': transaction_day.indices.keys(),
   'total_number': transaction_day.size()['size'],
   'average_amount': transaction_day.aggregate({'amount': 'mean'})['amount']
})

`Much less transactions amount occurs over the weekend.` Let's see how weekend activity compares with that during weekday 

In [10]:
if 'day_type' not in transactions:
   transactions['day_type'] = transactions['day_name'].apply(lambda day_name: 'weekend' if day_name == 'Saturday' or day_name == 'Sunday' else 'business day')

In [ ]:
sns.kdeplot(data=transactions, x='amount', hue='day_type');

The distribution is heavily right-skewed so it's hard to determine any notable hidden patterns. Let's apply a logarithmic transformation to normalize it 

In [11]:
transactions['log_amount'] = transactions.amount.map(lambda amount: np.log(amount))

In [ ]:
transactions_by_daytype = sns.FacetGrid(data=transactions, col='day_type', height=5)
transactions_by_daytype.map(sns.histplot, 'log_amount', kde=True, bins=20);

It looks like there are two notable groups of customer activities during week day: One group is revolved about `e^5 ≈ $148.41`; another group revolves about `e^7 ≈ $1096.63`

Let's see what types of transactions occur during the weekend

In [ ]:
import seaborn.objects as so

In [ ]:
sns.displot(transactions, x='log_amount', hue='txn_description', multiple='stack');

In [ ]:
pd.DataFrame(transactions.groupby(['day_name', 'txn_description']).amount.size()).unstack().plot(kind='bar', stacked=True, figsize=(15,8));
plt.title('Number of Transactions Each Day by Desription');
plt.legend(['INTER BANK', 'PAY/SALARY', 'PAYMENT', 'PHONE BANK', 'POS', 'SALES-POS']);
plt.xticks(rotation='horizontal');
plt.ylabel('Number of Transactions');

The second group of more expensive activity turns out to be salary payment. And all salary activities took place during week day only so it make sense that less transactions of high amount occurred during weekend.

## Export Dataset for Second Task

In [13]:
transactions.to_csv(f'./data/ANZ_clean_transactions.csv')